In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tkinter import Tk, filedialog
import os


In [5]:
# Data directories
train_dir = 'data/train'
val_dir = 'data/val'

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    'data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3561 images belonging to 5 classes.
Found 888 images belonging to 5 classes.


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=10
)

model.save('insect_classifier.h5')


C:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\Asus\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


111/111 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - accuracy: 0.3069 - loss: 1.5852 - val_accuracy: 0.4514 - val_loss: 1.3334
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4375 - loss: 1.2667 - val_accuracy: 0.3750 - val_loss: 1.3760
Epoch 3/10


C:\Users\Asus\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


111/111 ━━━━━━━━━━━━━━━━━━━━ 95s 832ms/step - accuracy: 0.5299 - loss: 1.1547 - val_accuracy: 0.4988 - val_loss: 1.2113
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5625 - loss: 1.0600 - val_accuracy: 0.5833 - val_loss: 1.2704
Epoch 5/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 99s 870ms/step - accuracy: 0.6039 - loss: 0.9967 - val_accuracy: 0.6157 - val_loss: 1.0227
Epoch 6/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7812 - loss: 0.6766 - val_accuracy: 0.7083 - val_loss: 0.9741
Epoch 7/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 94s 830ms/step - accuracy: 0.6772 - loss: 0.8316 - val_accuracy: 0.6019 - val_loss: 1.0410
Epoch 8/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5938 - loss: 0.8260 - val_accuracy: 0.3333 - val_loss: 1.4725
Epoch 9/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 95s 836ms/step - accuracy: 0.7198 - loss: 0.7297 - val_accuracy: 0.6574 - val_loss: 1.0682
Epoch 10/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7812 - loss: 0.6330 - val_accuracy: 

In [ ]:

model = tf.keras.models.load_model('insect_classifier.h5')
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    return np.argmax(prediction, axis=1)[0]

def classify_real_time():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        img_array = cv2.resize(frame, (150, 150)) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        prediction = model.predict(img_array)
        class_idx = np.argmax(prediction, axis=1)[0]
        class_label = list(train_generator.class_indices.keys())[class_idx]
        
        cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Real-Time Insect Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def upload_and_classify():
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    if file_path:
        class_idx = classify_image(file_path)
        class_label = list(train_generator.class_indices.keys())[class_idx]
        print(f'The uploaded image is classified as: {class_label}')
        
        img = cv2.imread(file_path)
        cv2.putText(img, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Uploaded Image Classification', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == '__main__':
    choice = input("Choose 'r' for real-time detection or 'u' to upload an image: ").strip().lower()
    if choice == 'r':
        classify_real_time()
    elif choice == 'u':
        upload_and_classify()
    else:
        print("Invalid choice!")


Choose 'r' for real-time detection or 'u' to upload an image:  r


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import tensorflow as tf

class App:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        
        self.label = tk.Label(self.window, text="Choose an option:")
        self.label.pack(pady=10)
        
        self.camera_button = tk.Button(self.window, text="Open Camera", width=20, command=self.open_camera)
        self.camera_button.pack(pady=5)
        
        self.upload_button = tk.Button(self.window, text="Upload Image", width=20, command=self.upload_image)
        self.upload_button.pack(pady=5)
        
        self.model = self.load_model()  # Load the animal emotion detection model
        
        self.window.mainloop()

    def load_model(self):
        try:
            model = tf.keras.models.load_model('insect_classifier.h5')
            return model
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load model: {e}")
            return None

    def open_camera(self):
        self.window.withdraw()  # Hide the main window temporarily
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "Failed to open camera.")
            self.window.deiconify()  # Restore the main window
            return
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Perform inference on the frame
            emotions, frame = self.detect_emotion(frame)
            
            cv2.imshow('Camera', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        self.window.deiconify()  # Restore the main window

    def upload_image(self):
        self.window.withdraw()  # Hide the main window temporarily
        
        file_path = filedialog.askopenfilename()
        if not file_path:
            self.window.deiconify()  # Restore the main window
            return
        
        try:
            image = cv2.imread(file_path)
            emotions, image = self.detect_emotion(image)
            self.display_image(image)
            messagebox.showinfo("Insect Detected", f"Emotions: {emotions}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to open image: {e}")
        
        self.window.deiconify()  # Restore the main window

    def detect_emotion(self, image):
        try:
            resized_image = cv2.resize(image, (224, 224))  # Resize to match model input size
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)  # Convert to RGB
            resized_image = np.expand_dims(resized_image, axis=0)  # Add batch dimension
            
            # Preprocess image if necessary (e.g., normalize pixel values)
            # Perform prediction
            predictions = self.model.predict(resized_image)
            
            # Process predictions to get emotions (example code, adjust as per your model's output)
            # For example, assuming the model outputs a softmax activation for emotions:
            emotions = ["Butterfly", "Dragonfly", "Grasshopper", "Ladybird", "Mosquito"]
            max_index = np.argmax(predictions)
            predicted_emotion = emotions[max_index]
            
            # Example overlay text on image
            cv2.putText(image, predicted_emotion, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            return predicted_emotion, image
        except Exception as e:
            messagebox.showerror("Error", f"Error detecting emotion: {e}")
            return None, image

    def display_image(self, image):
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        
        self.image_label = tk.Label(self.window, image=image)
        self.image_label.image = image
        self.image_label.pack(padx=10, pady=10)
        
        close_button = tk.Button(self.window, text="Close", command=self.close_image)
        close_button.pack(pady=5)

    def close_image(self):
        self.image_label.pack_forget()

if __name__ == "__main__":
    window = tk.Tk()
    app = App(window, "Animal Emotion Detection System")